# Costas Loop



## Initialization of variables

In [5]:
import numpy as np
import matplotlib.pyplot as plt

# Initializations

phi_hat = 0.0      # phi_hat is the output phase 
delta_phi = 0.0    # delta_phi is the phase difference between the incomming signal and the VCO
wn = 0.01          # wn is omega_n, pll bandwidth
zeta = 0.707       # Damping factor of the loop
K = 1000           # K = loop gain, which I assume can be chosen, but this is a sample value

# These are just initializing some vectors for plotting
ref = np.linspace(0, 400, 400)
out = []


# The ax, tx, and bx variables are for the loop parameters. Using feedback theory I derived (ok, I stole)
# an equation in the form (a0*z^-1 + a1*z^-1 + a2*z^-1)/(b0*z^-1 + b1*z^-1 + b2*z^-1). tx are just other
# ways of expressing the variables wn, zeta, and K
a0 = 1.0
a1 = -2.0
a2 = 1.0

t1 = K/(wn*wn)     # tau_1
t2 = 2*zeta/wn     # tau_2

b0 = (4*K/t1) * (1.0+t2/2.0)
b1 = 8*K/t1
b2 = (4*K/t1) * (1.0-t2/2.0)


# In order for the filter desing to work a buffer is needed to cycle values through
v0 = 0.0           # Buffer for the filter values
v1 = 0.0           # Buffer for the filter values
v2 = 0.0           # Buffer for the filter values

## Main portion of the code

In [ ]:
# Main portion of the code

# pll = Costas_Loop()

for i in range(400):
    phi = 0.0              # Original signal phase
    signal_in = np.exp = (1j * phi) # Complex input signal
    vco = np.exp(1j * phi_hat) # Complex oscillator signal
    delta_phi = np.angle( signal_in * np.conj( vco))
    ref.append(signal_in)
    output.append(vco)
    
    v2 =  v1
    v1 =  v0
    v0 =  delta_phi - v1* a1 - v2* a2
    phi_hat =  v0 * b0 + v1 * b1 + v2* b2
    
plt.plot(ref)
plt.plot(out)
#plt.plot(diff)
plt.show()